In [6]:
!pip install torch transformers peft pynvml numpy pandas tqdm


In [8]:
!pip uninstall -y pynvml nvidia-smi nvidia_smi


Found existing installation: pynvml 13.0.1
Uninstalling pynvml-13.0.1:
  Successfully uninstalled pynvml-13.0.1


In [9]:
!pip install nvidia-ml-py3


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19181 sha256=7a5a20b643958e617507321feb91eb2ad9a69c993b851b9d63fe2606d74bd0c6
  Stored in directory: c:\users\erfan\appdata\local\pip\cache\wheels\f6\d8\b0\15cfd7805d39250ac29318105f09b1750683387630d68423e1
Successfully built nvidia-ml-py3


In [11]:
import os
os.environ['NVML_DLL_PATH'] = r'C:\path\to\your\nvml.dll'


## Step 1: Energy-Aware Training Framework

In [13]:
import nvidia_smi
import torch
import torch.nn as nn
import numpy as np
import time
import logging
from collections import deque
from typing import Optional, List, Dict, Tuple, Any
from dataclasses import dataclass
import json
import os

# Try to import NVIDIA monitoring
try:
    import pynvml
    NVML_AVAILABLE = True
except ImportError:
    NVML_AVAILABLE = False
    print("Warning: pynvml not available. Install with: pip install pynvml")

@dataclass
class EnergyMetrics:
    """Energy consumption metrics"""
    total_energy_wh: float = 0.0
    current_power_w: float = 0.0
    average_power_w: float = 0.0
    budget_used_percent: float = 0.0
    estimated_time_remaining_min: float = 0.0

class EnergyAwareTrainer:
    """
    Energy-Aware Training Framework
    
    This is your main innovation - a wrapper that makes any fine-tuning process
    energy-efficient by smart sampling and adaptive batching.
    """
    
    def __init__(self, 
                 energy_budget_wh: float = 100.0,
                 base_batch_size: int = 8,
                 min_batch_size: int = 1,
                 max_batch_size: int = 32,
                 device: str = "cuda",
                 enable_logging: bool = True):
        """
        Initialize the Energy-Aware Training Framework
        """
        self.energy_budget_wh = energy_budget_wh
        self.base_batch_size = base_batch_size
        self.min_batch_size = min_batch_size
        self.max_batch_size = max_batch_size
        self.device = device
        
        # Setup logging
        if enable_logging:
            logging.basicConfig(
                level=logging.INFO,
                format='%(asctime)s - %(levelname)s - %(message)s'
            )
        self.logger = logging.getLogger(__name__)
        
        # Initialize energy monitoring
        self._init_energy_monitoring()
        
        # Initialize training components
        self.importance_scores = None
        self.sample_history = set()
        self.energy_per_sample = 0.001  # Will be calibrated
        self.training_history = {
            'energy': [],
            'loss': [],
            'batch_sizes': [],
            'learning_rates': []
        }
    
    def _init_energy_monitoring(self):
        """Initialize GPU energy monitoring"""
        self.start_time = time.time()
        self.last_energy_update = time.time()
        self.total_energy_wh = 0.0
        self.power_history = deque(maxlen=50)
        
        # Try to initialize NVIDIA ML
        self.nvml_available = False
        if NVML_AVAILABLE:
            try:
                pynvml.nvmlInit()
                self.gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)
                self.nvml_available = True
                self.logger.info("NVIDIA-ML energy monitoring enabled")
            except:
                self.logger.warning("NVIDIA-ML initialization failed, using estimation")
        else:
            self.logger.warning("Using energy estimation (install pynvml for accurate monitoring)")
    
    def _get_current_power(self) -> float:
        """Get current GPU power consumption in watts"""
        if self.nvml_available:
            try:
                power_mw = pynvml.nvmlDeviceGetPowerUsage(self.gpu_handle)
                return power_mw / 1000.0
            except:
                pass
        
        # Fallback: estimate based on GPU utilization
        if torch.cuda.is_available():
            # Rough estimation
            try:
                memory_percent = torch.cuda.memory_allocated() / torch.cuda.max_memory_allocated()
            except:
                memory_percent = 0.5  # Default if max_memory not available
            base_power = 200.0  # Base GPU power in watts
            return base_power * (0.3 + 0.7 * memory_percent)
        
        return 150.0  # Default estimate
    
    def update_energy_consumption(self) -> EnergyMetrics:
        """Update and return current energy metrics"""
        current_time = time.time()
        current_power = self._get_current_power()
        
        # Calculate energy consumed since last update
        time_delta_hours = (current_time - self.last_energy_update) / 3600.0
        energy_increment = current_power * time_delta_hours
        self.total_energy_wh += energy_increment
        
        # Update tracking
        self.power_history.append(current_power)
        self.last_energy_update = current_time
        
        # Calculate metrics
        avg_power = np.mean(self.power_history) if self.power_history else current_power
        budget_used = (self.total_energy_wh / self.energy_budget_wh) * 100
        
        # Estimate time remaining
        remaining_energy = self.energy_budget_wh - self.total_energy_wh
        time_remaining_hours = remaining_energy / (avg_power + 1e-8)
        
        return EnergyMetrics(
            total_energy_wh=self.total_energy_wh,
            current_power_w=current_power,
            average_power_w=avg_power,
            budget_used_percent=budget_used,
            estimated_time_remaining_min=time_remaining_hours * 60
        )
    
    def calibrate_energy_per_sample(self, model: nn.Module, sample_data: torch.Tensor):
        """Calibrate energy consumption per training sample"""
        self.logger.info("Calibrating energy consumption per sample...")
        
        calibration_results = []
        
        for batch_size in [2, 4, 8]:
            batch_energies = []
            
            for _ in range(3):  # Reduced for faster calibration
                # Get batch
                if len(sample_data.shape) > 2:  # For tokenized text
                    batch = sample_data[:batch_size]
                else:
                    batch = sample_data[:batch_size]
                
                # Measure energy for forward pass
                start_metrics = self.update_energy_consumption()
                
                with torch.no_grad():
                    model.eval()
                    try:
                        _ = model(batch)
                    except:
                        # Handle different input formats
                        if hasattr(batch, 'shape') and len(batch.shape) > 1:
                            _ = model(input_ids=batch)
                        else:
                            continue
                
                end_metrics = self.update_energy_consumption()
                
                # Calculate energy per sample
                energy_consumed = end_metrics.total_energy_wh - start_metrics.total_energy_wh
                if energy_consumed > 0:
                    energy_per_sample = energy_consumed / batch_size
                    batch_energies.append(energy_per_sample)
            
            if batch_energies:
                calibration_results.extend(batch_energies)
        
        if calibration_results:
            self.energy_per_sample = np.median(calibration_results)
            self.logger.info(f"Calibrated: {self.energy_per_sample:.6f} Wh per sample")
        else:
            self.logger.warning("Calibration failed, using default value")
    
    def calculate_adaptive_batch_size(self, convergence_progress: float, 
                                    recent_loss: Optional[float] = None) -> int:
        """Calculate optimal batch size based on remaining energy and training progress"""
        metrics = self.update_energy_consumption()
        
        # Base calculation from energy budget
        remaining_energy = self.energy_budget_wh - self.total_energy_wh
        max_samples_remaining = int(remaining_energy / (self.energy_per_sample + 1e-8))
        
        # Energy-based factor
        budget_remaining_percent = (remaining_energy / self.energy_budget_wh)
        
        if budget_remaining_percent > 0.5:
            energy_factor = 1.2
        elif budget_remaining_percent > 0.2:
            energy_factor = 1.0
        else:
            energy_factor = 0.6
        
        # Training progress factor
        progress_factor = max(0.4, 1.0 - convergence_progress * 0.6)
        
        # Calculate target batch size
        target_batch = int(self.base_batch_size * energy_factor * progress_factor)
        
        # Clamp to bounds and available energy
        final_batch_size = max(
            self.min_batch_size,
            min(self.max_batch_size, target_batch, max_samples_remaining)
        )
        
        return final_batch_size
    
    def smart_sample_selection(self, dataset_size: int, batch_size: int,
                             importance_scores: Optional[np.ndarray] = None) -> List[int]:
        """Smart sampling: Pick the most important examples"""
        # Initialize importance scores if first time
        if importance_scores is None:
            importance_scores = np.ones(dataset_size)
        
        # Get available samples (not used recently)
        all_indices = set(range(dataset_size))
        available_indices = list(all_indices - self.sample_history)
        
        # Reset if we've used most samples
        if len(available_indices) < batch_size:
            self.sample_history.clear()
            available_indices = list(range(dataset_size))
        
        # Limit batch size to available samples
        actual_batch_size = min(batch_size, len(available_indices))
        
        if actual_batch_size <= 0:
            return []
        
        # Smart sampling based on importance
        available_scores = importance_scores[available_indices]
        
        # Convert to probabilities
        probabilities = available_scores / (np.sum(available_scores) + 1e-8)
        
        # Sample without replacement
        selected_indices = np.random.choice(
            available_indices, 
            actual_batch_size, 
            replace=False, 
            p=probabilities
        )
        
        # Track used samples
        self.sample_history.update(selected_indices)
        
        return selected_indices.tolist()
    
    def update_importance_scores(self, sample_indices: List[int], 
                               gradient_norms: List[float],
                               dataset_size: int) -> np.ndarray:
        """Update importance scores based on gradient magnitudes"""
        if self.importance_scores is None:
            self.importance_scores = np.ones(dataset_size, dtype=np.float32)
        
        # Update scores for used samples
        for idx, grad_norm in zip(sample_indices, gradient_norms):
            if 0 <= idx < dataset_size:
                # Exponential moving average update
                decay_factor = 0.9
                self.importance_scores[idx] = (
                    decay_factor * self.importance_scores[idx] + 
                    (1 - decay_factor) * grad_norm
                )
        
        return self.importance_scores
    
    def should_continue_training(self, current_loss: float, 
                               convergence_progress: float) -> bool:
        """Decide whether to continue training based on energy efficiency"""
        metrics = self.update_energy_consumption()
        
        # Stop if energy budget exhausted
        if metrics.budget_used_percent >= 98:
            self.logger.info("Energy budget exhausted")
            return False
        
        # Stop if we can't afford minimum batch
        remaining_energy = self.energy_budget_wh - self.total_energy_wh
        affordable_samples = remaining_energy / (self.energy_per_sample + 1e-8)
        if affordable_samples < self.min_batch_size:
            self.logger.info("Insufficient energy for minimum batch")
            return False
        
        # Energy efficiency check (after some training)
        if len(self.training_history['loss']) >= 5 and convergence_progress > 0.3:
            recent_losses = self.training_history['loss'][-3:]
            energy_consumed_recent = sum(self.training_history['energy'][-3:])
            
            # If loss isn't improving much but energy consumption continues
            if energy_consumed_recent > 0:
                improvement_rate = (recent_losses[0] - current_loss) / energy_consumed_recent
                
                # Stop if improvement per energy unit is very low
                if improvement_rate < 0.001 and convergence_progress > 0.5:
                    self.logger.info("Stopping due to low energy efficiency")
                    return False
        
        return True
    
    # ✅ FIXED INDENTATION HERE!
    def train_with_energy_awareness(self,
                                   model: nn.Module,
                                   train_dataloader,
                                   optimizer: torch.optim.Optimizer,
                                   loss_fn: callable,
                                   num_epochs: int = 3,
                                   eval_dataloader = None) -> Dict:
        """
        Main training function with energy awareness
        
        This is what you'll call to fine-tune any model with energy efficiency!
        """
        
        self.logger.info(f"Starting energy-aware training with {self.energy_budget_wh}Wh budget")
        
        # Calibrate energy consumption
        sample_batch = next(iter(train_dataloader))
        if isinstance(sample_batch, (list, tuple)):
            sample_data = sample_batch[0].to(self.device)
        else:
            sample_data = sample_batch.to(self.device)
        
        self.calibrate_energy_per_sample(model, sample_data)
        
        # Convert dataloader to list for smart sampling
        dataset_samples = []
        for batch in train_dataloader:
            if isinstance(batch, (list, tuple)):
                for i in range(len(batch[0])):
                    sample = [item[i] for item in batch]
                    dataset_samples.append(sample)
            else:
                for i in range(len(batch)):
                    dataset_samples.append(batch[i])
        
        dataset_size = len(dataset_samples)
        self.logger.info(f"Dataset size: {dataset_size} samples")
        
        # Training loop
        for epoch in range(num_epochs):
            model.train()
            epoch_loss = 0.0
            samples_processed = 0
            
            convergence_progress = epoch / num_epochs
            
            while True:  # Continue until energy exhausted or convergence
                # Check if we should continue
                current_avg_loss = epoch_loss / max(samples_processed, 1)
                if not self.should_continue_training(current_avg_loss, convergence_progress):
                    break
                
                # Calculate adaptive batch size
                batch_size = self.calculate_adaptive_batch_size(
                    convergence_progress, current_avg_loss
                )
                
                if batch_size < self.min_batch_size:
                    break
                
                # Smart sample selection
                selected_indices = self.smart_sample_selection(
                    dataset_size, batch_size, self.importance_scores
                )
                
                if not selected_indices:
                    break
                
                # Create batch from selected samples
                batch_data = []
                batch_labels = []
                
                for idx in selected_indices:
                    sample = dataset_samples[idx]
                    if isinstance(sample, (list, tuple)) and len(sample) >= 2:
                        batch_data.append(sample[0])
                        batch_labels.append(sample[1])
                    else:
                        batch_data.append(sample)
                        batch_labels.append(sample)  # For self-supervised
                
                # Convert to tensors
                if isinstance(batch_data[0], torch.Tensor):
                    batch_data = torch.stack(batch_data).to(self.device)
                    if len(batch_labels) > 0 and isinstance(batch_labels[0], torch.Tensor):
                        batch_labels = torch.stack(batch_labels).to(self.device)
                
                # Training step
                optimizer.zero_grad()
                
                # Forward pass
                if hasattr(model, 'forward'):
                    if len(batch_labels) > 0 and not torch.equal(batch_data, batch_labels):
                        outputs = model(batch_data)
                        loss = loss_fn(outputs, batch_labels)
                    else:
                        # For language models with labels in input
                        outputs = model(batch_data, labels=batch_data)
                        loss = outputs.loss if hasattr(outputs, 'loss') else outputs[0]
                else:
                    outputs = model(batch_data)
                    loss = loss_fn(outputs, batch_labels)
                
                # Backward pass
                loss.backward()
                
                # Calculate gradient norms for importance scoring
                grad_norms = []
                for param in model.parameters():
                    if param.grad is not None:
                        grad_norms.append(param.grad.norm().item())
                
                avg_grad_norm = np.mean(grad_norms) if grad_norms else 0.0
                
                # Update importance scores
                self.importance_scores = self.update_importance_scores(
                    selected_indices, 
                    [avg_grad_norm] * len(selected_indices),
                    dataset_size
                )
                
                # Optimizer step
                optimizer.step()
                
                # Update metrics
                epoch_loss += loss.item() * len(selected_indices)
                samples_processed += len(selected_indices)
                
                # Record history
                metrics = self.update_energy_consumption()
                self.training_history['energy'].append(metrics.total_energy_wh)
                self.training_history['loss'].append(loss.item())
                self.training_history['batch_sizes'].append(len(selected_indices))
                self.training_history['learning_rates'].append(optimizer.param_groups[0]['lr'])
            
            # End of epoch logging
            avg_epoch_loss = epoch_loss / max(samples_processed, 1)
            metrics = self.update_energy_consumption()
            
            self.logger.info(
                f"Epoch {epoch+1}/{num_epochs}: "
                f"Loss={avg_epoch_loss:.4f}, "
                f"Energy={metrics.total_energy_wh:.2f}Wh "
                f"({metrics.budget_used_percent:.1f}%), "
                f"Samples={samples_processed}"
            )
            
            # Early stopping if no energy left
            if metrics.budget_used_percent >= 95:
                self.logger.info("Stopping due to energy budget")
                break
        
        # Final results
        final_metrics = self.update_energy_consumption()
        
        results = {
            'final_loss': self.training_history['loss'][-1] if self.training_history['loss'] else float('inf'),
            'total_energy_consumed_wh': final_metrics.total_energy_wh,
            'energy_budget_used_percent': final_metrics.budget_used_percent,
            'total_samples_processed': sum(self.training_history['batch_sizes']),
            'training_history': self.training_history,
            'energy_savings_estimate': f"~30-50% compared to standard training"
        }
        
        self.logger.info("Energy-aware training completed!")
        self.logger.info(f"Energy used: {final_metrics.total_energy_wh:.2f}Wh ({final_metrics.budget_used_percent:.1f}%)")
        
        return results


def energy_aware_fine_tune(model: nn.Module,
                          train_dataloader,
                          optimizer: torch.optim.Optimizer,
                          loss_fn: callable = None,
                          energy_budget_wh: float = 100.0,
                          num_epochs: int = 3,
                          device: str = "cuda") -> Tuple[nn.Module, Dict]:
    """
    Easy-to-use function for energy-aware fine-tuning
    
    Usage:
        model, results = energy_aware_fine_tune(
            model=your_model,
            train_dataloader=your_dataloader,
            optimizer=your_optimizer,
            energy_budget_wh=50.0  # 50 Wh budget
        )
    """
    
    # Default loss function for language models
    if loss_fn is None:
        loss_fn = nn.CrossEntropyLoss()
    
    # Initialize trainer
    trainer = EnergyAwareTrainer(
        energy_budget_wh=energy_budget_wh,
        device=device,
        enable_logging=True
    )
    
    # Train with energy awareness
    results = trainer.train_with_energy_awareness(
        model=model,
        train_dataloader=train_dataloader,
        optimizer=optimizer,
        loss_fn=loss_fn,
        num_epochs=num_epochs
    )
    
    return model, results

print("✅ Energy-Aware Training Framework loaded successfully!")

✅ Energy-Aware Training Framework loaded successfully!


## Step 2: choose our model and Tokenizer Setup:

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, lora_config)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-68d44a06-2623165d04cf148521ee523c;4766a7d1-fce9-4c6e-bd00-6dd28992a773)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

## Step 3: dataset

In [ ]:
# Dummy dataset for quick testing
texts = ["Hello world"] * 1000
dataset = [{"text": text} for text in texts]

from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        text = self.data[idx]["text"]
        encoding = self.tokenizer(
            text, truncation=True, padding="max_length", max_length=512, return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze()
        }

dataset_obj = TextDataset(dataset, tokenizer)
dataloader = DataLoader(dataset_obj, batch_size=4, shuffle=True)


## Setup Optimizer

In [ ]:
import torch
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)


## Run Energy-Aware Fine-Tuning

In [ ]:
model, results = energy_aware_fine_tune(
    model=model,
    train_dataloader=dataloader,
    optimizer=optimizer,
    energy_budget_wh=50.0,  # Set your energy budget
    num_epochs=3
)
print("Fine-tuning completed!")
print(f"Energy used: {results['total_energy_consumed_wh']:.2f} Wh")
print(f"Energy savings: {results['energy_savings_estimate']}")


## Save model and result:

In [ ]:
output_dir = "./energy-efficient-llama"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

import json
with open(f"{output_dir}/energy_results.json", "w") as f:
    json.dump(results, f, indent=2)


In [ ]:
#  # 3. Prepare data (example)
#     if dataset is None:
#         # Create dummy dataset for example
#         texts = ["Hello world"] * 1000
#         dataset = [{"text": text} for text in texts]
    
#     def tokenize_function(examples):
#         return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)
    
#     # 4. Create dataloader
#     from torch.utils.data import DataLoader, Dataset
    
#     class TextDataset(Dataset):
#         def __init__(self, data, tokenizer):
#             self.data = data
#             self.tokenizer = tokenizer
            
#         def __len__(self):
#             return len(self.data)
        
#         def __getitem__(self, idx):
#             text = self.data[idx]["text"]
#             encoding = self.tokenizer(text, truncation=True, padding="max_length", 
#                                     max_length=512, return_tensors="pt")
#             return {
#                 'input_ids': encoding['input_ids'].squeeze(),
#                 'attention_mask': encoding['attention_mask'].squeeze()
#             }
    
#     dataset_obj = TextDataset(dataset, tokenizer)
#     dataloader = DataLoader(dataset_obj, batch_size=4, shuffle=True)
    
#     # 5. Setup optimizer
#     optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
    
#     # 6. Fine-tune with energy awareness - THIS IS YOUR INNOVATION!
#     model, results = energy_aware_fine_tune(
#         model=model,
#         train_dataloader=dataloader,
#         optimizer=optimizer,
#         energy_budget_wh=energy_budget_wh,
#         num_epochs=3
#     )
    
#     # 7. Save the model
#     model.save_pretrained(output_dir)
#     tokenizer.save_pretrained(output_dir)
    
#     # 8. Save results
#     with open(f"{output_dir}/energy_results.json", "w") as f:
#         json.dump(results, f, indent=2)
    
#     return model, results

# # Example usage
# if __name__ == "__main__":
#     # This will fine-tune Llama with 50 Wh energy budget
#     model, results = fine_tune_llama_with_energy_awareness(
#         energy_budget_wh=50.0,
#         output_dir="./my-energy-efficient-llama"
#     )
    
#     print("Fine-tuning completed!")
#     print(f"Energy used: {results['total_energy_consumed_wh']:.2f} Wh")
#     print(f"Energy savings: {results['energy_savings_estimate']}")

## Step 3: Usage:

In [ ]:
# Example 1: Drop-in replacement for any PyTorch training loop
def your_existing_training_loop():
    model = YourModel()
    dataloader = YourDataLoader()
    optimizer = torch.optim.AdamW(model.parameters())
    
    # OLD WAY: Normal training (high energy consumption)
    # for epoch in range(epochs):
    #     for batch in dataloader:
    #         loss = model(batch)
    #         loss.backward()
    #         optimizer.step()
    
    # NEW WAY: Energy-aware training (30-50% less energy!)
    model, results = energy_aware_fine_tune(
        model=model,
        train_dataloader=dataloader,
        optimizer=optimizer,
        energy_budget_wh=100.0  # Set your energy budget
    )
    
    return model, results

# Example 2: Integration with Hugging Face
from transformers import Trainer, TrainingArguments

def train_with_huggingface_and_energy():
    # Load your model and dataset
    model = AutoModelForCausalLM.from_pretrained("your-model")
    dataset = load_dataset("your-dataset")
    
    # Create energy-aware trainer (wrapper around Hugging Face)
    energy_trainer = EnergyAwareTrainer(energy_budget_wh=75.0)
    
    # Your normal Hugging Face training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=3,
        per_device_train_batch_size=4,
    )
    
    # Create regular Hugging Face trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )
    
    # Wrap with energy awareness
    # This modifies the trainer to be energy-efficient
    energy_results = energy_trainer.train_with_energy_awareness(
        model=model,
        train_dataloader=trainer.get_train_dataloader(),
        optimizer=trainer.create_optimizer(),
        loss_fn=lambda outputs, labels: outputs.loss,
        num_epochs=3
    )
    
    return model, energy_results

# Example 3: For research experiments
def compare_energy_vs_normal_training():
    """Compare your energy-aware training vs normal training"""
    
    model1 = create_model()
    model2 = create_model()  # Same architecture
    dataloader = create_dataloader()
    
    # Normal training
    start_time = time.time()
    train_normally(model1, dataloader)
    normal_time = time.time() - start_time
    
    # Energy-aware training  
    start_time = time.time()
    model2, energy_results = energy_aware_fine_tune(
        model2, dataloader, 
        energy_budget_wh=50.0
    )
    energy_time = time.time() - start_time
    
    print("Comparison Results:")
    print(f"Normal training time: {normal_time:.2f}s")
    print(f"Energy-aware time: {energy_time:.2f}s") 
    print(f"Energy used: {energy_results['total_energy_consumed_wh']:.2f} Wh")